In [28]:
# Task:
# An SVD Recommender that predicts the rating a user will give to a movie
# based on the user's own ratings and other users' rating data.

# Use only 'rating' as the data, avoid 'tags' and 'genre'

# 80/20, train/test split. Additionally, do a temporal split. 


In [29]:
# imports
import pandas as pd
from numpy.linalg import svd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
import math
import random
import copy

In [30]:
def draw_ascii_percentage_bar(value):
    filled_length = int(value *  100)
    empty_length = 100 - filled_length

    bar = '[' + '%' * filled_length + ']'
    #  + '_' * empty_length
    
    print(bar, end="")

In [31]:
# read data
movies = 'data/movielens-latest-small/movies.csv'
ratings = 'data/movielens-latest-small/ratings.csv'

# to dataframes
df_movies = pd.read_csv(movies)
df_ratings = pd.read_csv(ratings)

# inspect them
display('Movies')
display(df_movies.head())
display('Ratings')
display(df_ratings.head())

'Movies'

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


'Ratings'

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
# 80/20, train/test split
df_ratings_x = df_ratings[['userId', 'movieId', 'timestamp']]
df_ratings_y = df_ratings[['rating', 'timestamp']]

x_train, x_test, y_train, y_test = train_test_split(df_ratings_x, df_ratings_y, test_size=0.2, random_state=1)
print(f"Training rows = {x_train.shape[0]}")
print(f"Testing rows = {x_test.shape[0]}")

#display(x_train.head())
#display(x_test.head())
#display(y_train.head())
#display(y_test.head())

# temporal split
tscv = TimeSeriesSplit(n_splits=2, test_size=20000)
for i, (train_index, test_index) in enumerate(tscv.split(x_train, y_train)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")

print(f"Training rows temporal split = {train_index.shape[0]}")
print(f"Testing rows temporal split = {test_index.shape[0]}")

Training rows = 80668
Testing rows = 20168
Fold 0:
  Train: index=[    0     1     2 ... 40665 40666 40667]
  Test:  index=[40668 40669 40670 ... 60665 60666 60667]
Fold 1:
  Train: index=[    0     1     2 ... 60665 60666 60667]
  Test:  index=[60668 60669 60670 ... 80665 80666 80667]
Training rows temporal split = 60668
Testing rows temporal split = 20000


In [6]:
# Consider reviews from users with more than 50 reviews
#usercount = df_ratings[['movieId','userId']].groupby('userId').count()
#display(usercount.head())

In [22]:
# Source for SVD stuff: https://machinelearningmastery.com/using-singular-value-decomposition-to-build-a-recommender-system/
# Build a pivot table with movieIds as columns 
# and users and their ratings as rows
rating_matrix = df_ratings.pivot(index="userId", columns="movieId", values="rating").fillna(0)
#display(rating_matrix.head())
matrix = rating_matrix.values
matrix = np.matrix(matrix)
display(matrix)
#matrix = np.mat([[5, 5, 3, 0, 5, 5], [5, 0, 4, 0, 4, 4], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]])
#display(matrix)
#matrix = np.mat([[4, 0, 4, 0, 0, 4], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]])
#display(matrix)


matrix([[4. , 0. , 4. , ..., 0. , 0. , 0. ],
        [0. , 0. , 0. , ..., 0. , 0. , 0. ],
        [0. , 0. , 0. , ..., 0. , 0. , 0. ],
        ...,
        [2.5, 2. , 2. , ..., 0. , 0. , 0. ],
        [3. , 0. , 0. , ..., 0. , 0. , 0. ],
        [5. , 0. , 0. , ..., 0. , 0. , 0. ]])

In [24]:
# Singular value decomposition
U, S, VT = np.linalg.svd(matrix.T, full_matrices=False)
# We know that the columns of vh are movies
# The rows of u are users

V = VT.T
Sigma = np.diag(S)

In [11]:
def cosine_similarity(v,u):
    return (v @ u)/ (np.linalg.norm(v) * np.linalg.norm(u))

def rmse_function(v, u):
    return np.sqrt(np.mean((v - u) ** 2))

def squared_distance(v, u):
    return np.sum((v - u) ** 2)

def manhattan_distance(v, u):
    return np.sum(np.abs(v - u))

In [12]:
# K-means RMSE Clustering 
# select k random data points (user indexes in u) as initial cluster centers C_1, ..., C_k

def initialize_centroids(data_points, k, seed = None):
    if seed is not None:
        random.seed(seed)
        
    # Select k random indexes from the data points list
    centroid_indexes = random.sample(range(data_points.shape[0]), k)
    
    # Construct the list of centroids using the selected indexes
    centroids = [copy.deepcopy(data_points[i]) for i in centroid_indexes]
    
    return centroids

def assign_point_to_centroid(user_index, point, centroid_index, assignments):
    if centroid_index not in assignments:
        assignments[centroid_index] = [(user_index, point)]
    else:
        assignments[centroid_index].append((user_index, point))

def update_centroid_position(centroid, new_position):
    centroid[:] = new_position
    
def average_point(points):
    # Extract the points (user points) from the tuples
    user_points = [point[1] for point in points]
    
    # Convert the list of points to a NumPy array
    points_array = np.array(user_points)

    # Compute the mean along each dimension
    average = np.mean(points_array, axis=0)
    
    return average

def k_means(u, k, max_iterations, seed, threshold):
    assignments = {}
    num_datapoints = u.shape[0]
    
    centroids = initialize_centroids(u, k, seed)

    for iteration in range(1, max_iterations):
        #print()
        #print(" Iteration ", i, end="")

        assignments = {}
        non_converged_centroids = copy.deepcopy(k)
        
        # for each p (user) in u, map p_i to its nearest cluster center C_j 
        for user_index, p in enumerate(u):
            # Find the closest centroid to the current data point
            #closest_centroid_index, _ = max(enumerate(centroids), key=lambda c: cosine_similarity(p, c[1]))
            closest_centroid_index, _ = max(enumerate(centroids), key=lambda c: cosine_similarity(p, c[1]))

            assign_point_to_centroid(user_index, p, closest_centroid_index, assignments)

        for index, centroid in enumerate(centroids):
            # Update centroid position by taking the mean of assigned data points
            
            assigned_points = assignments[index]  # get all data points assigned to a centroid
            if assigned_points:
                
                #draw_ascii_percentage_bar(len(assigned_points) / num_datapoints)
                #print(" C", index, ", ", len(assigned_points), " points")

                new_position = average_point(assigned_points)
                 # calculate Euclidean distance

                distance = np.linalg.norm(centroid - new_position)
                
                if distance < threshold:
                    non_converged_centroids -= 1

                update_centroid_position(centroid, new_position)
            
        if (non_converged_centroids == 0):
            print("iter: ", iteration, " ******* CONVERGED ******* ", end="")
            break
    
    total_rmse = 0
    for centroid_index, data_points in assignments.items():
        squared_distances = 0

        for point_index, point in enumerate(data_points):
            # calculate squared distancs of each point to its centroid
            squared_distances += squared_distance(point[1], centroids[centroid_index])
        
        # divide the sum of squared distances with the number of points in the centroid
        mse = squared_distances / len(centroids[centroid_index])

        # take the square root of mse to get rmse
        rmse = np.sqrt(mse)
        total_rmse += rmse
    
    # Calculate the average RMSE
    average_rmse = total_rmse / len(assignments)
    print("average_rmse: ", average_rmse)
    
    #centroids_and_assignments = [(centroids[i], assignments[i]) for i in range(len(centroids))]
    centroids_and_assignments = np.array([(centroids[i], assignments[i]) for i in range(len(centroids))], dtype=object)

    return centroids_and_assignments

In [25]:
max_iterations = 50
threshold = 1e-5

# Use first 2 singular values. Source: https://www.kaggle.com/code/vincentman0403/recommendation-example-by-svd
r = 2

# Get approximate U, Sigma, VT
Ur = U[:, :r]

#print("matrix.shape[0]: ", matrix.shape[0])

#print("Sigma: ", Sigma)

Sr = Sigma[:r, :r]
Vr = V[:, :r]

# use specific start conditions
k = 7
seed = 0

centroids_and_assignments = k_means(Ur, k, max_iterations, seed, threshold)


NameError: name 'k_means' is not defined

In [13]:
# Generate similarities to all others users for a single user
def get_similarities(target_point, neighbors):
    similarities = []

    for index, neighbor in enumerate(neighbors):
        similarity = cosine_similarity(target_point, neighbor[1])
        similarities.append((neighbor, similarity))
    
    return similarities

def predict_rating(centroid_and_assignment, movie_index, ratings_matrix, k=5):
    
    # Calculate cosine similarity between the centroid and its neighbors
    similarities = get_similarities(centroid_and_assignment[0], centroid_and_assignment[1])

    # Calculate weighted average of ratings from nearest neighbors
    weighted_ratings = 0
    total_similarity = 0
    for neighbor_index, similarity in enumerate(similarities):

        neighbor_similarity = similarity[1]
        neighbor_rating = ratings_matrix[similarity[0][0], movie_index]
        
        if neighbor_rating != 0:  # Ignore if neighbor hasn't rated the movie
            weighted_ratings += neighbor_similarity * neighbor_rating
            total_similarity += neighbor_similarity
            
            #print("neighbor_similarity: ", neighbor_similarity)
            #print("neighbor_rating: ", neighbor_rating)
            #print("weighted rating: ", neighbor_similarity * neighbor_rating)
            #print("total_similarity: ", total_similarity)
            #print()
            #break
    
    # Predict the rating for the target user
    if total_similarity != 0:
        predicted_rating = weighted_ratings / total_similarity
    else:
        predicted_rating = 0  # In case none of the nearest neighbors have rated the movie
    
    return predicted_rating

In [14]:
# Evaluate predictions

# for each user, predict rating for every movie

# first, find closest centroid
# then give prediction based on the centroid

# check centroid's distance from actual, if review exists

def find_closest_centroid(data_point, centroids):
    closest_centroid_distance = np.inf
    closest_centroid_index = None
    
    for centroid_index, centroid in enumerate(centroids):
        distance = squared_distance(data_point, centroid)
        if distance < closest_centroid_distance:
            closest_centroid_distance = distance
            closest_centroid_index = centroid_index
            
    return closest_centroid_index

for row in range(0, 100): #u.shape[0]):
    centroids = [centroid for centroid, _ in centroids_and_assignments]

    closest_centroid_index = find_closest_centroid(Ur[row], centroids)
    print("closest_centroid_index:", closest_centroid_index)
    
    predicted_rating = predict_rating(centroids_and_assignments[closest_centroid_index], movie_index = 0, ratings_matrix = matrix)
    print("Predicted rating:", predicted_rating)
    print()

    #for col in range(0, u.shape[1]):
        


closest_centroid_index: 3


IndexError: index 615 is out of bounds for axis 0 with size 610

In [27]:
def evaluate_one():
    # new user data point
    new = np.full((1, matrix.shape[1]), 3)
    newresult = new * Ur * np.linalg.inv(Sr)
    
    print('Vector of new user to latent factor = ', newresult)
    
    return
    
    unique_point = np.array([0.3, 0.8])
    
    closest_centroid_index = find_closest_centroid(unique_point, centroids)
    print("closest_centroid_index:", closest_centroid_index)

    predicted_rating = predict_rating(centroids_and_assignments[closest_centroid_index], movie_index = 1, ratings_matrix = matrix)
    print("Predicted rating:", predicted_rating)    

evaluate_one()

Vector of new user to latent factor =  [[-0.25971894 -0.01331114]]


In [ ]:
# Evaluate predictions

# for each user, predict rating for every movie
# check distance from actual, if review exists
total_error = 0

for row in range(0, u.shape[0]):
    user_total_error_squared = 0
    user_evaluations = 0
    
    for col in range(0, vh.shape[1]):
        predicted_rating = predict_rating(user_index = row, movie_index = col, ratings_matrix = matrix, u = u)
        actual_rating = matrix[row][col]
        if actual_rating != 0:
            user_evaluations += 1
            user_total_error_squared += (predicted_rating - actual_rating)**2
    
    user_total_error = math.sqrt(user_total_error_squared)
    print("User total error:", user_total_error)
    print("User evaluations:", user_evaluations)
    
print("Total error:", total_error)

In [ ]:
max_iterations = 50
threshold = 1e-5

# Find converging start conditions, and lowest rmse
for seed in range(0, 1):
    print("seed:",seed," ",end="")
    
    for k in range(3, 10): 
        print("k:",k," ",end="")
        k_means(u, k, max_iterations, seed, threshold)